<a href="https://colab.research.google.com/github/SoumyaShreeram/Microlensing_with_NeuralNets/blob/master/01_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Testing Neural Network Architecture

Author: Soumya Shreeram <br>
Script adapted from: Millon Martin & Kevin Müller <br>
Date: 23rd February 2020

Import all required python libraries

In [0]:
from google.colab import drive
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt
import random
import os
from IPython.display import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, InputSpec
from tensorflow.python.keras.layers import Conv1D, Conv2D
from tensorflow.python.keras.layers import MaxPooling1D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Add, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

Mount google drive

In [0]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Setting up the path to the data directory

In [0]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '/gdrive/My Drive/EPFL/training_set_microlensing/')

### 1. Input parameters

In [0]:
v_t = [500, 300] # transverse velcity
v_t_idx = 1 # choose index value for the velocity you would like (0 or 1)
r_0 = [2,4,10,15,20,30,40,60,80,100] # scale radius

In [0]:
def setSamplingParameters(v_t, euler_sampling, data_dir):
  """
  Function defines the values for sampling parameters and no. of pixels 

  Inputs:
  @v_t :: transverse velocity
  @euler_sampling :: sampling of real data taken by the Euler telescope
  @data_dir :: directory with the model light-curves for inputted v_t

  Retutns:
  @n_sample, n_sample_max :: ...
  @n_pix :: ...
  """
  if v_t == 500:
    if euler_sampling: 
      n_sample, n_sample_max, n_pix = 5000, 5000, 4546
    else:
      n_sample, n_sample_max, n_pix = 5000, 5000, 486

  else:
    n_sample, n_sample_max, n_pix = 20000, 3000, 1137
    sample_params = [n_sample, n_sample_max, n_pix]
  return sample_params

def initializer(r_0, sample_params):
  """
  Function defines the class names, categories, and initializes the data arrays
  Input:
  @r_0 :: arr with all the scale radii of the background quasar
  @sample_params :: arr containing values for sampling frequency 
  """
  # generate categories and class names 
  classes = np.array(map(str, r_0))
  categories = np.arange(len(r_0))

  # initialize data arrays to be classified
  in_data = np.zeros(sample_params[0]*len(r_0), sample_params[2], 1)
  out_catergories, out_radii = np.zeros(sample_params[0]*len(r_0))
  
  return in_data, out_catergories, out_radii

def getFilename(data_dir, r, v_t[v_t_idx]):
  if v == v_t[v_t_idx]:
    filename = data_dir + 'simLC_A-B_n20000_v300_R' + str(r)  + '_M0,3.pkl'
  else:
    filename = data_dir + 'simLC_A-B_n50000_v500_R' + str(r)  + '_M0,3.pkl'

In [0]:
# sets the sampling parameters
sample_params = setSamplingParameters(v_t[v_t_idx], euler_sampling, data_dir)

# initializes data arrays
in_data, out_catergories, out_radii = initializer(r_0, sample_params)

for r in range(len(r_0)):
  filename = getFilename(data_dir, r, v_t[v_t_idx])
  
  # load data from the file
  in_data, out_catergories, out_radii = loadData(filename)